# BEATRIZ CÁRDABA RICO
## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [524]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

In [525]:
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType

# Librería para dar formato a las salidas obtenidas
import pandas as pd
# Librería para aplicar expresiones regulares en el parseado
import re
# librería para parsear las fechas con formato date
from datetime import datetime
# Librería que permite descargar información de una url desde python
import urllib
from urllib.request import urlretrieve
# Librería para llamar a funciones relacionadas con el sistema operativo
import os

In [526]:
sc = SparkContext.getOrCreate()


In [527]:
sqlContext = SQLContext(sc)

Importamos los datos desde CSV textfil de spark:


In [528]:
indicadores_desarrollo = "indicadores_desarrollo.csv"
raw_desarrollo = sc.textFile(indicadores_desarrollo)

In [529]:
indicadores_deuda = "indicadores_deuda.csv"
raw_deuda = sc.textFile(indicadores_deuda)

# PARSEO
 -  DATOS DE DESARROLLO

In [530]:
raw_desarrollo.take(5) # visualizamos las 5 primeras filas

['country_name,country_code,anho,GC.DOD.TOTL.CN,GC.DOD.TOTL.GD.ZS,SP.DYN.CBRT.IN',
 'Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197',
 'Albania,ALB,2016 [YR2016],1188814490000,80.7355748396923,12.08',
 'Australia,AUS,2014 [YR2014],674700000,0.042207528166503,13.2']

Comprobamos que las columnas están separadas por comas, parseamos los datos de desarrollo:

In [531]:
dt_desarrollo = raw_desarrollo.map(lambda x: x.split(",")) # para separar los diferentes campos

Visualizamos que ahora las columnas se diferencian correctamente:




In [532]:
dt_desarrollo.take(2) # visualizamos las dos primeras filas y comprobamos que se diferencian las columnas correctamente

[['country_name',
  'country_code',
  'anho',
  'GC.DOD.TOTL.CN',
  'GC.DOD.TOTL.GD.ZS',
  'SP.DYN.CBRT.IN'],
 ['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259']]

- DATOS DEUDA

In [533]:
raw_deuda.take(5)  # visualizamos las 5 primeras filas

['country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD',
 'Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,',
 'Albania,ALB,54250280.6,790248675.2,514185620,182197616.7,236447897.3,0,39445139.5,170018.4,39615157.9,76050616.1,317194512.5,234321242.3,310371858.4,4542664.9,2279989.2,6822654.1,13847333.6,165602386.9,87884000,28101536.1,41948869.7,31030688.2,4618504.3,120324.7,35769517.2',
 'Algeria,DZA,95188724.6,171185188.1,75420000,0,95188724.6,,576463.5,0,576463.5,10320772.2,10320772.2,,10320772.2,,,,7680627.6,19031728.7,3220000,8094779,

Comprobamos que las columnas están separadas por comas, parseamos los datos de deuda:

In [534]:
dt_deuda = raw_deuda.map(lambda x: x.split(",")) # para separar los diferentes campos

In [535]:
dt_deuda.take(2) # visualizamos las dos primeras filas y comprobamos que se diferencian las columnas correctamente

[['country_name',
  'country_code',
  'DT.AMT.BLAT.CD',
  'DT.AMT.DLXF.CD',
  'DT.AMT.DPNG.CD',
  'DT.AMT.MLAT.CD',
  'DT.AMT.OFFT.CD',
  'DT.AMT.PBND.CD',
  'DT.AMT.PCBK.CD',
  'DT.AMT.PROP.CD',
  'DT.AMT.PRVT.CD',
  'DT.DIS.BLAT.CD',
  'DT.DIS.DLXF.CD',
  'DT.DIS.MLAT.CD',
  'DT.DIS.OFFT.CD',
  'DT.DIS.PCBK.CD',
  'DT.DIS.PROP.CD',
  'DT.DIS.PRVT.CD',
  'DT.INT.BLAT.CD',
  'DT.INT.DLXF.CD',
  'DT.INT.DPNG.CD',
  'DT.INT.MLAT.CD',
  'DT.INT.OFFT.CD',
  'DT.INT.PBND.CD',
  'DT.INT.PCBK.CD',
  'DT.INT.PROP.CD',
  'DT.INT.PRVT.CD'],
 ['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  '']]

##### 2. Número de países distintos en cada dataset. Coinciden?

 - Desarrollo:

En primer lugar vamos a comprobar los datos diferentes para la columna de country_name en los datos de desarrollo. Lo realizamos mediante un distict count. NO se ha eliminado la fila de la cabecera por lo que al resultado habrá que restarle 1

In [599]:
(dt_desarrollo.map(lambda line: line[0]).distinct().count())-1 # realizamos un distict count de los paises y eliminamos el valor de la cabecera pra que no lo cuente

51

In [611]:
countries_desarrollo = dt_desarrollo.map(lambda line: line[0]).\
                                     filter(lambda line: line[0] != 'country_name').\
                                     distinct()

In [612]:
countries_desarrollo.take(51)

['Albania',
 'Australia',
 '"Bahamas',
 'Barbados',
 'Belize',
 'Bhutan',
 'Botswana',
 'Brazil',
 'El Salvador',
 'Guatemala',
 'Hungary',
 'Ireland',
 'Kazakhstan',
 'Kyrgyz Republic',
 'Malawi',
 'Palau',
 'Peru',
 'Philippines',
 'Russian Federation',
 'Solomon Islands',
 'St. Kitts and Nevis',
 'St. Vincent and the Grenadines',
 'Switzerland',
 'Thailand',
 'Ukraine',
 'United States',
 'Uruguay',
 'Zambia',
 'country_name',
 'Belarus',
 'Colombia',
 'Georgia',
 'Indonesia',
 'Jamaica',
 'Japan',
 'Jordan',
 '"Korea',
 'Malaysia',
 '"Micronesia',
 'Moldova',
 'Namibia',
 'New Zealand',
 'Papua New Guinea',
 'San Marino',
 'Seychelles',
 'Singapore',
 'Spain',
 'Sri Lanka',
 'St. Lucia',
 'Tonga',
 'Turkey']

 - Deuda:

 En primer lugar vamos a comprobar los datos diferentes para la columna de country_name en los datos de desarrollo. Lo realizamos mediante un distict count. NO se ha eliminado la fila de la cabecera por lo que al resultado habrá que restarle 1

In [613]:
(dt_deuda.map(lambda line: line[0]).distinct().count())-1 # hacemos un count ditict de los paises y restamos 1 que viene de la cabecera

123

Eliminamos la cabecera de la columna country_name:

In [614]:
# queremos seleccionaar la columna 0 y de esta quere filtrar y que no aparezca la cabecera
countries_deuda = dt_deuda.map(lambda line: line[0]).\
                           filter(lambda line: line[0] != 'country_name').\
                           distinct()


In [615]:
countries_deuda.take(123) # visualizamos los 123 países de deuda

['Albania',
 'Algeria',
 'Angola',
 'Belize',
 'Bhutan',
 'Botswana',
 'Brazil',
 'Cabo Verde',
 'Cambodia',
 'China',
 'Costa Rica',
 "Cote d'Ivoire",
 'Ecuador',
 '"Egypt',
 'El Salvador',
 'Eritrea',
 'Ethiopia',
 'Fiji',
 'Gabon',
 '"Gambia',
 'Ghana',
 'Guatemala',
 'Guinea-Bissau',
 'Honduras',
 'India',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Kyrgyz Republic',
 'Least developed countries: UN classification',
 'Lebanon',
 'Liberia',
 'Malawi',
 'Maldives',
 'Mauritania',
 'Mauritius',
 'Mexico',
 'Montenegro',
 'Mozambique',
 'Myanmar',
 'Nepal',
 'Nigeria',
 'Pakistan',
 'Paraguay',
 'Peru',
 'Philippines',
 'Russian Federation',
 'Solomon Islands',
 'Somalia',
 'South Africa',
 'St. Vincent and the Grenadines',
 'Syrian Arab Republic',
 'Tanzania',
 'Thailand',
 'Timor-Leste',
 'Togo',
 'Tunisia',
 'Turkmenistan',
 'Ukraine',
 'Uzbekistan',
 '"Venezuela',
 'Vietnam',
 '"Yemen',
 'Zambia',
 'country_name',
 'Afghanistan',
 'Armenia',
 'Azerbaijan',
 'Bangladesh',
 'Belarus',
 'Ben

- Solución 2:

En esta pregunat concluimos que el data set desarrollo contine 51 países diferentes y el datset de duda contiene 123 países diferenets

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

Lo que vamos a realixar en este ejercicio es un map apara seleccionar las columnas en las que estamos interesados. En primer lugar nos interesa la columna country_name, que ocupa la posición 0 y en segundo lugar nos interesa la columna DT.AMT.MLAT.CD, que ocupa la posición 5. Ambas en el data set de deuda.

En el parseo de datos y comprobando el dataset deuda en el csv, vista excel, se ha visto que se produce un error ya que hay países que en la primera columna "country_name" tienen una coma, por lo que su parseo es erróneo. Comporbamos que países tienen su parseo erróneo visualizzando las tres primeras columnas del data set deuda:


In [622]:
dt_deuda.map(lambda x: (x[0], x[1], x[2])).collect()

[('country_name', 'country_code', 'DT.AMT.BLAT.CD'),
 ('Afghanistan', 'AFG', '61739336.9'),
 ('Albania', 'ALB', '54250280.6'),
 ('Algeria', 'DZA', '95188724.6'),
 ('Angola', 'AGO', '8473824016.3'),
 ('Armenia', 'ARM', '68968314.7'),
 ('Azerbaijan', 'AZE', '106369937'),
 ('Bangladesh', 'BGD', '386702219.8'),
 ('Belarus', 'BLR', '1487363278.5'),
 ('Belize', 'BLZ', '23329498.4'),
 ('Benin', 'BEN', '26014505.8'),
 ('Bhutan', 'BTN', '189877869.8'),
 ('Bolivia', 'BOL', '67814879.9'),
 ('Bosnia and Herzegovina', 'BIH', '108704966.2'),
 ('Botswana', 'BWA', '7903563.5'),
 ('Brazil', 'BRA', '2097380010.6'),
 ('Bulgaria', 'BGR', '35764202.8'),
 ('Burkina Faso', 'BFA', '27371336'),
 ('Burundi', 'BDI', '10461498.5'),
 ('Cabo Verde', 'CPV', '11767654.3'),
 ('Cambodia', 'KHM', '223300273.6'),
 ('Cameroon', 'CMR', '9999925153.3'),
 ('Central African Republic', 'CAF', '8462062.9'),
 ('Chad', 'TCD', '82357214.9'),
 ('China', 'CHN', '6532446441.9'),
 ('Colombia', 'COL', '447557894.1'),
 ('Comoros', 'COM'

Los países pobrematicos son aquellos que en su columna [2] tienen el código del país en vez de una cifra. Tras observar que esto no sucede en muchos países vamos a eliminar éstos de la muestra para llegar al objetivo del ejercicio, proporcionar la relación entre cada país y la deuda

In [659]:

# seleccionamos las columnas que nos interesn, en este caso la posicion 0 y 1 para identificar al país  y la 5.
# como he comentado anteriormente vamos a eliminar de la miestra los países mal parseados porque sino apareciría como deuda DT.AMT.MLAT.CD otra columna diferente

dt_deuda_3 = dt_deuda.map(lambda x: (x[0], x[1],x[2], x[5])).\
                filter(lambda x: x[1] != '').\
                filter(lambda x: x[1] != ("") ).\
                filter(lambda x: x[2] != ("COD") ).\
                filter(lambda x: x[2] != ("COG") ).\
                filter(lambda x: x[2] != ("EGY") ).\
                filter(lambda x: x[2] != ("MKD") ).\
                filter(lambda x: x[2] != ("VEN") ).\
                filter(lambda x: x[2] != ("YEM") ).\
                filter(lambda x: x[2] != ('IRN') ).\
                filter(lambda x: x[2] != ("GMB") )

In [663]:
dt_deuda_3.map(lambda x: (x[0], x[3])).collect()

[('country_name', 'DT.AMT.MLAT.CD'),
 ('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Angola', '124688691.9'),
 ('Armenia', '94331207.1'),
 ('Azerbaijan', '513623656.5'),
 ('Bangladesh', '1039564682.7'),
 ('Belarus', '595438826'),
 ('Belize', '29683326.5'),
 ('Benin', '91631639.5'),
 ('Bhutan', '17390420.5'),
 ('Bolivia', '357721211.5'),
 ('Bosnia and Herzegovina', '290170946.5'),
 ('Botswana', '125652344.5'),
 ('Brazil', '2514318741.6'),
 ('Bulgaria', '228993019.8'),
 ('Burkina Faso', '119510893'),
 ('Burundi', '16115111.8'),
 ('Cabo Verde', '39970124.9'),
 ('Cambodia', '72307001.8'),
 ('Cameroon', '152095968.1'),
 ('Central African Republic', '2241813.8'),
 ('Chad', '95738709.5'),
 ('China', '2615723714.1'),
 ('Colombia', '1100907553.6'),
 ('Comoros', '2182053.1'),
 ('Costa Rica', '272808260.7'),
 ("Cote d'Ivoire", '84629406.5'),
 ('Djibouti', '34254909.2'),
 ('Dominica', '11130355.8'),
 ('Dominican Republic', '254728306.1'),
 ('Ecuador', '578894793.9'

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD.

Para conseguir el resuultado de esta cuestión se va a proceder con diferentes procedimientos:

- Primero vamos a seleccionar las filas de estos indicadorees de deuda, que están en la posición 2,11 y 18.


Como en el ejercicio anterior, como hay países que no se han parseado correctamente los eliminamos de la muestra. Seleccionamos las filas que nos interesan. en este caso si no hubieramos eliminado bien las filas el resultado de la media de DT.AMT.BLAT.CD no se podría realizar porquie habría elementos str

In [697]:
# seleccionamos las columnas que nos intereasn en este caso y eliminamos las filas que han dado error en el parseo

dt_deuda_filter_4 = dt_deuda.map(lambda x: (x[0], x[1], x[2], x[11], x[18])).\
                filter(lambda x: x[1] != '').\
                filter(lambda x: x[1] != ("") ).\
                filter(lambda x: x[2] != ("COD") ).\
                filter(lambda x: x[2] != ("COG") ).\
                filter(lambda x: x[2] != ("EGY") ).\
                filter(lambda x: x[2] != ("MKD") ).\
                filter(lambda x: x[2] != ("VEN") ).\
                filter(lambda x: x[2] != ("YEM") ).\
                filter(lambda x: x[2] != ('IRN') ).\
                filter(lambda x: x[2] != ("GMB") )

dt_deuda_filter_4.take(200) # visualizamos que el filtro es correcto 

[('country_name',
  'country_code',
  'DT.AMT.BLAT.CD',
  'DT.DIS.BLAT.CD',
  'DT.INT.BLAT.CD'),
 ('Afghanistan', 'AFG', '61739336.9', '49114729.4', '39903620.1'),
 ('Albania', 'ALB', '54250280.6', '76050616.1', '13847333.6'),
 ('Algeria', 'DZA', '95188724.6', '10320772.2', '7680627.6'),
 ('Angola', 'AGO', '8473824016.3', '8838256901.1', '1005053965.1'),
 ('Armenia', 'ARM', '68968314.7', '174269846.7', '24094832'),
 ('Azerbaijan', 'AZE', '106369937', '84748079.6', '10085950'),
 ('Bangladesh', 'BGD', '386702219.8', '6141785637.5', '309079773.4'),
 ('Belarus', 'BLR', '1487363278.5', '2265336386.7', '630234914.9'),
 ('Belize', 'BLZ', '23329498.4', '3581448.7', '3455608.4'),
 ('Benin', 'BEN', '26014505.8', '334698287.3', '13208408.6'),
 ('Bhutan', 'BTN', '189877869.8', '95819437.4', '184064524.8'),
 ('Bolivia', 'BOL', '67814879.9', '554128686.3', '32609012.4'),
 ('Bosnia and Herzegovina', 'BIH', '108704966.2', '114375048.3', '12605397.7'),
 ('Botswana', 'BWA', '7903563.5', '1316332.2', '88

- Para poder calcular la media hay que eliminar las cabeceras de cada columna:

In [698]:
header = dt_deuda_filter_4.take(1)[0] # nombramos las cabecera
header # visualizamos que ha seleccionado correctamente la cabecera

('country_name',
 'country_code',
 'DT.AMT.BLAT.CD',
 'DT.DIS.BLAT.CD',
 'DT.INT.BLAT.CD')

In [699]:
# eliminamos del rdd la cabecera pra poder realizar cáculos numericos con las columnas
dt_deuda_filter_4 = dt_deuda_filter_4 \
    .filter(lambda line: line!=header) # selecciona todas las filas menos header

In [705]:
dt_deuda_filter_4.take(1) # comprobamos que la  primera fila no es el header

[('Afghanistan', 'AFG', '61739336.9', '49114729.4', '39903620.1')]

- Tenemos que pasar todas las columnas a tipo float para poder operar con ellas, Para poder hacerlo hay que eliminar aquellas filas que tengan valores nulos:

In [729]:
dt_deuda_filter_clean_4 = dt_deuda_filter_4.map(lambda x: (x[0], x[1], x[2], x[3], x[4])).\
                filter(lambda x: x[2] != '').\
                filter(lambda x: x[2] != ("")).\
                filter(lambda x: x[3] != '').\
                filter(lambda x: x[3] != ("")).\
                filter(lambda x: x[4] != '').\
                filter(lambda x: x[4] != ("") )



Nos aseguramos que todos los datos sobre los que se van a arealizar los cáculos son numéricos mediante la siguiente función:

In [730]:
dt_deuda_filter_clean_4 = dt_deuda_filter_clean_4.map(lambda x: (str(x[0]), str(x[1]), float(x[2]), float(x[3]), float(x[4]) ))


- Calculamos las medias de cada columna:

### Solución

- Media: 'DT.AMT.BLAT.CD'

In [731]:
dt_deuda_filter_clean_4.map(lambda x: (x[2])).mean() # DT.AMT.BLAT.CD'

691089509.5199999

   - Media: 'DT.DIS.BLAT.CD'

In [732]:
dt_deuda_filter_clean_4.map(lambda x: (x[3])).mean() # 'DT.DIS.BLAT.CD'

1241875121.030476

  - Media: 'DT.INT.BLAT.CD'

In [733]:
dt_deuda_filter_clean_4.map(lambda x: (x[4])).mean() # 'DT.INT.BLAT.CD'

179530290.29714286

### 5. Los 20 paises con DT.AMT.DLXF.CD más alto. Columna: DT.AMT.DLXF.CD 


En primer lugar seleccionamos las columnas que vamos a usar y eliminamos las filas que se han parseado mal

In [734]:
dt_deuda_filter_5 = dt_deuda.map(lambda x: (x[0], x[1], x[3])).\
                filter(lambda x: x[1] != '').\
                filter(lambda x: x[1] != ("") ).\
                filter(lambda x: x[1] != ("COD") ).\
                filter(lambda x: x[1] != ("COG") ).\
                filter(lambda x: x[1] != ("EGY") ).\
                filter(lambda x: x[1] != ("MKD") ).\
                filter(lambda x: x[1] != ("VEN") ).\
                filter(lambda x: x[1] != ("YEM") ).\
                filter(lambda x: x[1] != ('IRN') ).\
                filter(lambda x: x[1] != ("GMB") )

In [764]:
dt_deuda_filter_5.take(5) # comprobamos que se carga

[('country_name', 'country_code', 'DT.AMT.DLXF.CD'),
 ('Afghanistan', 'AFG', '100847181.9'),
 ('Albania', 'ALB', '790248675.2'),
 ('Algeria', 'DZA', '171185188.1'),
 ('Angola', 'AGO', '11067045628.1')]

A continucación:

- Seleccionamos las columnas a estudiar, posicion 0 y 2.
- Eliminamos las filas con valores no infados.
- eliminamos el header (otra manera a la del ejercicio anterior)

In [767]:
dt_deuda_DT_AMT_DLXF_CD = dt_deuda_filter_5.map(lambda x: (x[0], x[2])).filter(lambda x: x[1] != '').filter(lambda x: x[0] != 'country_name')

In [768]:

dt_deuda_DT_AMT_DLXF_CD.take(5) # comporbamos que se ha filtado correctamente

[('Afghanistan', '100847181.9'),
 ('Albania', '790248675.2'),
 ('Algeria', '171185188.1'),
 ('Angola', '11067045628.1'),
 ('Armenia', '845630723.8')]

Cambiamos el tipo de los datos para que se puedan ordenar:

In [748]:
dt_deuda_DT_AMT_DLXF_CD = dt_deuda_DT_AMT_DLXF_CD.\
        map(lambda x: (str(x[0]), float(x[1])))

## Solución:

Ordenamos según el valor de la columna 1, DT_AMT_DLXF_CD, de mayor a menor y seleccionamos las 20 primeras

In [749]:
(dt_deuda_DT_AMT_DLXF_CD.sortBy(lambda x: x[1])
    .zipWithIndex()
    .map(lambda x: x[0])
    .sortBy(lambda x: x[1], ascending= False)
).take(20)

[('China', 96218620835.7),
 ('Brazil', 90041840304.1),
 ('Russian Federation', 66589761833.5),
 ('Turkey', 51555031005.8),
 ('South Asia', 48756295898.2),
 ('India', 31923507000.8),
 ('Indonesia', 30916112653.8),
 ('Kazakhstan', 27482093686.4),
 ('Mexico', 25218503927.0),
 ('Least developed countries: UN classification', 25197029299.4),
 ('IDA only', 20483289208.0),
 ('Romania', 14013783350.4),
 ('Colombia', 11985674438.7),
 ('Angola', 11067045628.1),
 ('Cameroon', 10404814960.2),
 ('Lebanon', 9506919669.6),
 ('South Africa', 9474257551.9),
 ('Vietnam', 8873505909.2),
 ('Pakistan', 8336013891.3),
 ('Ukraine', 8148995625.6)]

##### 6. Pais con los datos informados todos los años.

In [769]:
dt_desarrollo.take(2) # visualizamos los datos desarrollo para ver las variables y el orden que tienen

[['country_name',
  'country_code',
  'anho',
  'GC.DOD.TOTL.CN',
  'GC.DOD.TOTL.GD.ZS',
  'SP.DYN.CBRT.IN'],
 ['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259']]

Vemos los diferentes años posibles:

In [770]:
years_desarrollo = dt_desarrollo.map(lambda line: line[2]).distinct()
years_desarrollo.take(10)

['2014 [YR2014]',
 '2015 [YR2015]',
 'KOR',
 'FSM',
 'anho',
 '2016 [YR2016]',
 'BHS',
 '2017 [YR2017]']

Hay 4 años disponibles. Seleccionaremos estos países agrupando los datos por país y sumamos cuantas veces aparece cada país. Nos interesan los paises que aparezzcan 4 veces

In [776]:
dt_desarrollo.map(lambda x: (x[0], x[2]))\
    .groupByKey()\
    .mapValues(lambda vals: len((vals)))\
    .sortByKey()\
    .filter(lambda x: x[1] ==4)\
    .collect()


[('Malawi', 4)]

Malawi es el único país que tiene informado los 4 años 

##### 7. Media anual de los distintos indicadores de desarrollo

En primer lugar como nos interesa calcular la media de los datos de desarrollo vamos a quitar las cabeceras para poder hjacer los calculos

In [777]:
header = dt_desarrollo.take(1)[0]
header

['country_name',
 'country_code',
 'anho',
 'GC.DOD.TOTL.CN',
 'GC.DOD.TOTL.GD.ZS',
 'SP.DYN.CBRT.IN']

In [778]:
# Seleccionamos la primera fila como la cabecera 
header = dt_desarrollo.take(1)[0]
    # Eliminamos la cabecera del rdd para poder trabajar con el
dt_desarrollo_nh= dt_desarrollo \
    .filter(lambda line: line!=header)

In [779]:
dt_desarrollo_nh.take(2) # comporbamos que hemos eliminado la cabecera

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197']]

En el ejercicio anterior al visualizar os datos hemos observado que hay tres filas que no se han parseado correctamente y hay que eliminarlas para poder realizar las medias.
Seleccionamos los datos que nos interesan y filtramos:

In [780]:
dt_desarrollo_filter_7 = dt_desarrollo_nh.map(lambda x: (x[0], x[2], x[3], x[4], x[5])).\
                filter(lambda x: x[1] != '').\
                filter(lambda x: x[1] != ("") ).\
                filter(lambda x: x[1] != ("KOR") ).\
                filter(lambda x: x[1] != ("FSM") ).\
                filter(lambda x: x[1] != ("BHS") )
                


In [784]:
dt_desarrollo_filter_7.take(5) # visualizamos que se han filtardo los datos correctamente

[('Albania', '2014 [YR2014]', '1023040800000', '73.3202469198429', '12.259'),
 ('Albania', '2015 [YR2015]', '1145500940000', '79.8644448747259', '12.197'),
 ('Albania', '2016 [YR2016]', '1188814490000', '80.7355748396923', '12.08'),
 ('Australia', '2014 [YR2014]', '674700000', '0.042207528166503', '13.2'),
 ('Australia', '2015 [YR2015]', '762718000', '0.0469480198522591', '12.9')]

Cambiamos el formato de las columnas:

 Indicamos que la columna 1 es un dato  que nos explica el año, y el resto de columnas son de formato int

In [782]:
dt_desarrollo_filter_7.map(lambda l: (year(l[1]),  int(l[2]), int(l[3]), int(l[4]), int(l[5])))

PythonRDD[1858] at RDD at PythonRDD.scala:53

In [786]:
dt_desarrollo_filter_7.take(5)

[('Albania', '2014 [YR2014]', '1023040800000', '73.3202469198429', '12.259'),
 ('Albania', '2015 [YR2015]', '1145500940000', '79.8644448747259', '12.197'),
 ('Albania', '2016 [YR2016]', '1188814490000', '80.7355748396923', '12.08'),
 ('Australia', '2014 [YR2014]', '674700000', '0.042207528166503', '13.2'),
 ('Australia', '2015 [YR2015]', '762718000', '0.0469480198522591', '12.9')]

 Seleccionamos la primera fila de la que queremos obtener la media:

In [788]:
dt_desarrollo_GC_DOD_TOTL_CN = dt_desarrollo_filter_7.map(lambda x: (x[1], x[2])).\
                filter(lambda x: x[1] != '')

Definimos una función según lo dado en el notebok Logs NASA Arturo Sanchez. Sin embargo, viendo el resultado me he dado cuenta que lo que hace es un count para cada año y no la media como interesaba en ete caso

In [789]:
unique_result = (dt_desarrollo_GC_DOD_TOTL_CN.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (dt_desarrollo_GC_DOD_TOTL_CN.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df = pd.DataFrame(day_count_dct )


In [790]:
day_count_df

,Día,Media
0,2014 [YR2014],47.0
1,2015 [YR2015],41.0
2,2016 [YR2016],36.0
3,2017 [YR2017],1.0


Repetimos el procedimiento para el segundo indicador:

In [ ]:
dt_desarrollo_GC_DOD_TOTL_GD_ZS = dt_desarrollo_filter.map(lambda x: (x[1], x[3])).\
                filter(lambda x: x[1] != '')

In [ ]:
unique_result = (dt_desarrollo_GC_DOD_TOTL_GD_ZS.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (dt_desarrollo_GC_DOD_TOTL_GD_ZS.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df_2 = pd.DataFrame(day_count_dct )

In [ ]:
day_count_df_2

Repetimos el procedimiento para el tercer indicador:

In [ ]:
dt_desarrollo_SP_DYN_CBRT_IN = dt_desarrollo_filter.map(lambda x: (x[1], x[4])).\
                filter(lambda x: x[1] != '')

In [ ]:
unique_result = (dt_desarrollo_SP_DYN_CBRT_IN.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (dt_desarrollo_SP_DYN_CBRT_IN.map(lambda line:  ((line[0]),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df_3 = pd.DataFrame(day_count_dct )

In [ ]:
day_count_df_3

Como esta función no ha servido para el resultado que pedía el ejercicio  he decidido parar los datos a formato tabla sql:

In [791]:
desarrollo_DF = sqlContext.createDataFrame(dt_desarrollo_filter) # convertimos el rdd en un tabla sql

In [792]:
desarrollo_DF.show(10) # observamos la tabla sql y las nuevas cabeceras:

+---------+-------------+----------------+------------------+------+
|       _1|           _2|              _3|                _4|    _5|
+---------+-------------+----------------+------------------+------+
|  Albania|2014 [YR2014]|   1023040800000|  73.3202469198429|12.259|
|  Albania|2015 [YR2015]|   1145500940000|  79.8644448747259|12.197|
|  Albania|2016 [YR2016]|   1188814490000|  80.7355748396923| 12.08|
|Australia|2014 [YR2014]|       674700000| 0.042207528166503|  13.2|
|Australia|2015 [YR2015]|       762718000|0.0469480198522591|  12.9|
|Australia|2016 [YR2016]|       907555000| 0.054648482520169|  12.9|
| Barbados|2014 [YR2014]|     12175704025|  129.629591844133|10.916|
| Barbados|2015 [YR2015]|     13365027241|  141.413895259761|10.806|
|  Belarus|2014 [YR2014]|     19781184418|  24.5487262642116|  12.5|
|  Belarus|2015 [YR2015]|33804450453.3197|  37.5981780556757|  12.5|
+---------+-------------+----------------+------------------+------+
only showing top 10 rows



 - Correspondecia de las columnas con las cabeceras
 
 _3 = 'GC.DOD.TOTL.CN',

 _4 ='GC.DOD.TOTL.GD.ZS',

 _5 = 'SP.DYN.CBRT.IN'

Se calcula la media de los valores de cada columna agrupando por año:


In [793]:
from pyspark.sql.functions import mean # importamos la fórmula para calcular la media

(desarrollo_DF
    .groupBy(("_2")) # agrupamos por año
    .agg(mean("_3"),mean("_4"),mean("_5")) # hacemos la media de cada columna 
    .show())

+-------------+--------------------+-----------------+------------------+
|           _2|             avg(_3)|          avg(_4)|           avg(_5)|
+-------------+--------------------+-----------------+------------------+
|2016 [YR2016]|  1.5364155866984E14|60.44188301621869|16.236916666666666|
|2014 [YR2014]|9.790764954429488E13|58.15214676864469|17.719702127659573|
|2017 [YR2017]| 2.23985861996269E12|  48.659564238322|            34.593|
|2015 [YR2015]|1.263079791448871E14|59.76292583345262|16.904780487804878|
+-------------+--------------------+-----------------+------------------+



##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

Para realizar este ejercico aprovechamos los cálculos y transformaciones del ejercico anterior. Ahora creamos una tabla en formto sql que contenga los 10 paises con un valor en media menor en el indicador SP.DYN.CBRT.IN.

Crecamos la tabla que asigne a cada país el valor medio de las observaciones del indicador P.DYN.CBRT.IN:

In [794]:

desarrollo_DF_sol = (desarrollo_DF
    .groupBy(("_1")) # queremos que agrupe por páis
    .agg(mean("_5")) # que calcule la media de la columna para cada país
    .sort(F.col("avg(_5)"))) # queremos orden ascendente

In [795]:
desarrollo_DF_sol.show(10) # visualizamos la tabla creada y qué 10 países necesitams para ver su deuda acomulada

+-----------+------------------+
|         _1|           avg(_5)|
+-----------+------------------+
|      Japan| 7.933333333333334|
| San Marino|              8.05|
|      Spain|               9.0|
|    Hungary| 9.533333333333333|
|  Singapore| 9.633333333333333|
|Switzerland|10.466666666666667|
|    Ukraine|              10.6|
|    Moldova|10.604333333333335|
|   Barbados|            10.861|
|   Thailand|           10.9175|
+-----------+------------------+
only showing top 10 rows



Creamos una tabla que contenga el nombre del pais y la deuda que queremos estudair qu eestá en la posión 5, además eliminamos la cabecera:

In [796]:
dt_deuda_DT_AMT_MLAT_CD= dt_deuda.map(lambda x: (x[0], x[5])).\
                filter(lambda x: x[0] != 'country_name')

Cambiamos el formato a tabla sql para que tenga el mismo formato que la tabla con los paises anteriores

In [797]:
dt_deuda_DT_AMT_MLAT_CD_DF = sqlContext.createDataFrame(dt_deuda_DT_AMT_MLAT_CD) # cambiamos a formato sql

In [798]:
dt_deuda_DT_AMT_MLAT_CD_DF.show(10) # visualizamos la forma de la tabla

+-----------+------------+
|         _1|          _2|
+-----------+------------+
|Afghanistan|    39107845|
|    Albania| 182197616.7|
|    Algeria|           0|
|     Angola| 124688691.9|
|    Armenia|  94331207.1|
| Azerbaijan| 513623656.5|
| Bangladesh|1039564682.7|
|    Belarus|   595438826|
|     Belize|  29683326.5|
|      Benin|  91631639.5|
+-----------+------------+
only showing top 10 rows



Unimos las tablas para ver para cada pais el valor SP.DYN.CBRT.IN y el valor DT_AMT_MLAT_CD_DF. en este caso he elegido una join left ya que quiero para cada valor de la tabla 1 su valor DT_AMT_MLAT_CD_DF.

In [799]:
data_join = desarrollo_DF_sol.join(dt_deuda_DT_AMT_MLAT_CD_DF,"_1","left")

In [800]:
data_join.take(1) # visualizamos que se ha credado correctamente

[Row(_1='Philippines', avg(_5)=22.935, _2='739056427.3')]

In [802]:
# Creamos una tabla con los paises que nos interesan a partir de la los 10 menores en SP.DYN.CBRT.IN 
data_join.createOrReplaceTempView("join_table")
    
    # Hacemos un WHERE para que nos vuelve solo aquellos paises que nos interesan:
join_df = sqlContext.sql("""
                SELECT _1, 'avg(_5)', _2
                FROM join_table 
                WHERE( _1 = 'Japan' OR
               _1 = 'San Marino' OR
                 _1 = 'Spain' OR
                 _1 = 'Hungary' OR
                 _1 = 'Singapore' OR
                _1 = 'Switzerland' OR
                 _1 = 'Ukraine' OR
                 _1 = 'Moldova' OR
                 _1 = 'Barbados' OR
                 _1 = 'Thailand')
                """)
join_df.show(10)

+-----------+-------+------------+
|         _1|avg(_5)|          _2|
+-----------+-------+------------+
|  Singapore|avg(_5)|        null|
| San Marino|avg(_5)|        null|
|      Spain|avg(_5)|        null|
|   Barbados|avg(_5)|        null|
|   Thailand|avg(_5)|  39071571.5|
|    Ukraine|avg(_5)|1072472830.3|
|Switzerland|avg(_5)|        null|
|      Japan|avg(_5)|        null|
|    Hungary|avg(_5)|        null|
|    Moldova|avg(_5)| 135842865.5|
+-----------+-------+------------+



del resultado observamos que muchos de estos paises no están recoguidos en la tabla deuda, nos quedamos con los únicos dos países que sí estaán:

In [806]:
joindf_def= sqlContext.sql("""
                SELECT _1, 'avg(_5)', _2
                FROM table4 
                WHERE( _1 = 'Ukraine' OR
                 _1 = 'Moldova' OR
                 _1 = 'Thailand')
                """)

#### Solución:

Sumamos los valores de estospaíes para DT_AMT_MLAT_CD_DF

In [ ]:
joindf_def.select(sum("_2")).show()

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?